In [2]:
import pandas as pd

btc_file_path = "bitcoin_historical_data_cleaned.csv"
btc_data = pd.read_csv(btc_file_path)
btc_data.describe()


,Close,High,Low,Open,Volume
count,2232.000000,2232.000000,2232.000000,2232.000000,2.232000e+03
mean,32720.074108,33397.097056,31939.574220,32680.525314,3.085689e+10
std,23705.349337,24194.384984,23145.630811,23677.004904,1.946871e+10
min,3399.470000,3427.950000,3391.020000,3401.380000,4.324201e+09
25%,10689.750000,10896.460000,10447.137500,10677.445000,1.803917e+10
50%,27794.805000,28305.765000,27245.495000,27781.830000,2.703812e+10
75%,47286.945000,48476.105000,46444.472500,47291.032500,3.818185e+10
max,106146.270000,109114.880000,105291.730000,106147.300000,3.509679e+11
